Load in required libraries, below.

In [1]:
# data 
import numpy as np
from sklearn.model_selection import train_test_split


## SageMaker Resources

The below cell stores the SageMaker session and role (for creating estimators and models), and creates a default S3 bucket. After creating this bucket, locally stored data can be uploaded to S3.

In [2]:
# sagemaker
import boto3
import sagemaker
from sagemaker import get_execution_role

In [3]:
# SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# default S3 bucket
bucket = sagemaker_session.default_bucket()
prefix='cnn-wendy-data-2b'
prefix_output='cnn-wendy-model-2b'

Here we retrieve the dataset of images and we upload it to S3


In [4]:
#!wget -nc https://da-youtube-ml.s3.eu-central-1.amazonaws.com/wendy-cnn/frames/wendy_cnn_frames_data_2b.zip
#!unzip -qq -n wendy_cnn_frames_data_2b.zip -d wendy_cnn_frames_data_2b 

--2020-10-25 23:18:01--  https://da-youtube-ml.s3.eu-central-1.amazonaws.com/wendy-cnn/frames/wendy_cnn_frames_data_2b.zip
Resolving da-youtube-ml.s3.eu-central-1.amazonaws.com (da-youtube-ml.s3.eu-central-1.amazonaws.com)... 52.219.75.100
Connecting to da-youtube-ml.s3.eu-central-1.amazonaws.com (da-youtube-ml.s3.eu-central-1.amazonaws.com)|52.219.75.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1094263352 (1.0G) [application/zip]
Saving to: ‘wendy_cnn_frames_data_2b.zip’

wendy_cnn_frames_da 100%[===================>]   1.02G  94.6MB/s    in 11s     

2020-10-25 23:18:12 (94.9 MB/s) - ‘wendy_cnn_frames_data_2b.zip’ saved [1094263352/1094263352]



In [6]:

# upload to S3. Skip if already uploaded. This can take a while.
#print('Uploading data to {}'.format(prefix))
#input_data = sagemaker_session.upload_data(path='wendy_cnn_frames_data_2b', bucket=bucket, key_prefix=prefix)
#print('Data uploaded to {}'.format(input_data))

Uploading data to cnn-wendy-data-2b
Data uploaded to s3://sagemaker-eu-central-1-283211002347/cnn-wendy-data-2b


In [4]:
# location to input data can be written down here, if known
input_data='s3://sagemaker-eu-central-1-283211002347/cnn-wendy-data-2b'

After uploading images to S3, we can define and train the estimator


In [5]:
# import a PyTorch wrapper
from sagemaker.pytorch import PyTorch

# specify an output path

output_path = 's3://{}/{}'.format(bucket, prefix_output)
print('Output path for models is {}'.format(output_path))

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir='letsplay_classifier',
                    role=role,
                    framework_version='1.6',
                    train_instance_count=1,
                    train_instance_type='ml.p2.8xlarge',
                    train_volume_size = 10,
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'img-width': 320,
                        'img-height': 180,
                        'batch-size': 16,
                        'layer-cfg': 'B',
                        'epochs': 5
                    })

Output path for models is s3://sagemaker-eu-central-1-283211002347/cnn-wendy-model-2b


## Train the Estimator

After instantiating the estimator, we train it with a call to `.fit()`. 

In [ ]:
%%time 
# train the estimator on S3 training data
estimator.fit({'train': input_data})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-10-26 00:49:31 Starting - Starting the training job...
2020-10-26 00:49:33 Starting - Launching requested ML instances.........
2020-10-26 00:51:28 Starting - Preparing the instances for training..

In [ ]:
print(estimator.model_data)
model_data = estimator.model_data
#model_data = 's3://sagemaker-eu-central-1-283211002347.s3.eu-central-1.amazonaws.com/cnn-wendy-model-2b/pytorch-training-2020-10-25-04-52-37-645/output/model.tar.gz'

We set up a model that can predict the class of an image

### Deploy the trained model

We deploy our model to create a predictor. We'll use this to make predictions on our data and evaluate the model.

In [ ]:
# importing PyTorchModel
from sagemaker.pytorch import PyTorchModel

# Create a model from the trained estimator data
# And point to the prediction script
model = PyTorchModel(model_data=model_data,
                     role = role,
                     framework_version='1.6',
                     entry_point='predict.py',
                     source_dir='letsplay_classifier')

In [ ]:
%%time
# deploy and create a predictor
              
predictor = model.deploy(initial_instance_count=1, instance_type='ml.p2.xlarge')


In [ ]:
# the endpoint where the predictor is located
endpoint_name = predictor.endpoint
print(endpoint_name)

Now that the model is deployed, we check how the predictor performs on our full dataset,
ensuring that the predictions make sense. We produce a classification report.


In [4]:

#endpoint_name='pytorch-inference-2020-10-25-06-03-13-786'

In [4]:

from sklearn.metrics import classification_report
from letsplay_classifier.endpoint import evaluate
y_true, y_pred = evaluate(endpoint_name, 'wendy_cnn_frames_data_2b', 0.15)


KeyboardInterrupt: 

In [2]:
report = classification_report(y_true=y_true, y_pred=y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       927
           1       0.96      0.97      0.96       183
           2       0.99      0.98      0.99      4808
           3       0.94      0.97      0.96        35
           4       0.97      0.97      0.97      1050

    accuracy                           0.98      7003
   macro avg       0.96      0.98      0.97      7003
weighted avg       0.98      0.98      0.98      7003



In [3]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred)

array([[ 916,    0,    7,    1,    3],
       [   2,  177,    0,    1,    3],
       [  45,    8, 4733,    0,   22],
       [   0,    0,    1,   34,    0],
       [   2,    0,   31,    0, 1017]])

In [4]:
!wget -nc https://da-youtube-ml.s3.eu-central-1.amazonaws.com/wendy-cnn/frames/wendy_cnn_frames_E67.zip
!unzip -qq -n wendy_cnn_frames_E67.zip -d wendy_cnn_frames_E67

--2020-10-25 15:59:24--  https://da-youtube-ml.s3.eu-central-1.amazonaws.com/wendy-cnn/frames/wendy_cnn_frames_E67.zip
Resolving da-youtube-ml.s3.eu-central-1.amazonaws.com (da-youtube-ml.s3.eu-central-1.amazonaws.com)... 52.219.75.88
Connecting to da-youtube-ml.s3.eu-central-1.amazonaws.com (da-youtube-ml.s3.eu-central-1.amazonaws.com)|52.219.75.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76964860 (73M) [application/zip]
Saving to: ‘wendy_cnn_frames_E67.zip’

wendy_cnn_frames_E6 100%[===================>]  73.40M  97.8MB/s    in 0.8s    

2020-10-25 15:59:25 (97.8 MB/s) - ‘wendy_cnn_frames_E67.zip’ saved [76964860/76964860]



In [5]:
from letsplay_classifier.interval.predict_intervals_endpoint import evaluate
evaluate(endpoint_name, 'wendy_cnn_frames_E67/E67', class_names= ['Battle', 'Hideout', 'Other', 'Siege', 'Tournament'])

23:04-28:04 | Battle : 8% , Hideout : 16% , Other : 7% , Siege : 65% 
46:28-47:42 | Battle : 32% , Other : 17% , Tournament : 49% 
51:02-51:04 | ????? 
52:30-53:02 | Battle : 70% , Hideout : 11% , Other : 13% 
54:36-56:00 | Battle : 38% , Other : 9% , Tournament : 48% 
01:03:52-01:05:42 | Battle : 25% , Other : 10% , Tournament : 62% 
01:14:00-01:16:36 | Tournament : 91% 
01:17:50-01:19:16 | Battle : 25% , Hideout : 7% , Other : 8% , Tournament : 58% 
01:33:12-01:34:22 | Battle : 82% , Other : 8% , Siege : 8% 
01:38:18-01:43:50 | Battle : 32% , Hideout : 7% , Other : 12% , Tournament : 46% 
01:43:54-01:46:06 | Battle : 16% , Other : 11% , Tournament : 70% 
01:49:00-01:50:38 | Battle : 92% , Other : 7% 
01:50:48-01:53:32 | Battle : 93% , Other : 6% 
01:55:52-01:57:46 | Battle : 87% , Tournament : 6% 


## Delete the Endpoint

Finally, I've added a convenience function to delete prediction endpoints after we're done with them. 

In [42]:
# Accepts a predictor endpoint as input
# And deletes the endpoint by name
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=endpoint_name)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [43]:
# delete the predictor endpoint 
delete_endpoint(predictor)

Deleted pytorch-inference-2020-10-23-05-01-33-314
